<a href="https://colab.research.google.com/github/bbberylll/ESAA/blob/main/%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 사용한 데이터 & 주제 설명
대회 주제: 은행 고객이 target=1인 거래를 했는지를 예측 (이진 분류 문제)
데이터 특성:
1. 익명화된 200개의 변수 (var_0 ~ var_199)
2. 각 row는 하나의 고객 정보
3. 불균형 데이터: target=1 비율이 매우 낮음

## 기법 적용
1. 모델: LightGBM (Gradient Boosting 기반 모델)
2. 전처리: 간단한 통계 기반 EDA 후 바로 모델 학습
3. Validation: Stratified K-Fold Cross Validation (5-fold)

예측 결과 평균을 이용해 submission 생성

## 문제 접근 방식
단순하고 강력한 방식으로 접근:

EDA는 가볍게 진행
복잡한 feature engineering 없이 raw feature 그대로 사용
모델 성능 향상을 위해 cross validation 반복 + 평균 ensemble

**정확도보다 AUC(Area Under Curve) 향상에 초점**

## 코드 흐름 요약


### (1) EDA
```
train.describe()
train['target'].value_counts(normalize=True)
train.corr()['target'].sort_values(ascending=False)
```


데이터 분포, 타겟 불균형 확인
변수 간 상관관계는 거의 없음 → 익명화된 feature 간 해석 불가

### (2) Feature Engineering & Processing

```
# 별도 Feature Engineering 없음
# 모든 var_ 변수를 그대로 사용
X = train.drop(['target', 'ID_code'], axis=1)
y = train['target']
```

### (3) 모델링 (LightGBM + StratifiedKFold)

```
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=44000)
for train_idx, valid_idx in folds.split(X, y):
    lgb_train = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx])
    lgb_valid = lgb.Dataset(X.iloc[valid_idx], label=y.iloc[valid_idx])
    
    clf = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_valid],
                    early_stopping_rounds=100, verbose_eval=100)
```

### (4) 예측 후 앙상블

```
predictions += clf.predict(test_X, num_iteration=clf.best_iteration) / folds.n_splits
```

## 차별점 (유의미한 포인트)

1. 복잡한 Feature Engineering 없이도 좋은 성능 달성

  → raw feature와 LGBM만으로도 고성능이 가능하다는 점을 증명

2. 불균형 데이터에서 Stratified K-Fold 적용

  → target=1 비율이 매우 적어 class 분포를 고려한 validation 필수

3. Ensemble 기법 없이도 단순 평균으로 안정성 확보

## 배울 점 요약

1. 접근 방식	: EDA와 모델링 사이의 균형, Overfitting 방지를 위한 early stopping
2. 모델 선택	: Tabular 데이터에 적합한 LightGBM 사용
3. Validation	: 불균형 데이터일수록 Stratified KFold의 필요성 증가
4. 단순함	: 불필요한 변수 생성보다 정제된 CV + 튜닝이 더 중요할 수 있음